## Extracting features from deep neural networks

This is the template I use to extract features from vision models. Features are saved as a .pkl dictionary where keys are the layer names (string) and values are the feature matrices (numpy array).

In [3]:
import sys
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms
import pandas as pd
import os
import pickle
from collections import OrderedDict
from PIL import Image
print("Packages loaded.")

Packages loaded.


In [4]:
## This line is for the HPC. 
# os.environ['TORCH_HOME'] = '/home/mualla/.cache/torch'

In [5]:
def get_layer_activations(model, layer_names, image_tensor):
    """
    Get the activations of specified layers in response to input data, handling large batches by
    splitting them into smaller batches of size 100, and concatenating the results. The activations
    are detached from the computation graph and moved to the CPU before storage.

    Args:
        model (torch.nn.Module): The neural network model to probe.
        layer_names (list): List of names of the layers to probe.
        image_tensor (torch.Tensor): Batch of images to feed through the model.

    Returns:
        dict: A dictionary where keys are layer names and values are concatenated activations for all batches,
              with each tensor detached and moved to CPU.
    """
    # Ensure layer_names is a list
    if not isinstance(layer_names, list):
        layer_names = [layer_names]

    activations = {name: [] for name in layer_names}
    hooks = []

    def get_activation(name):
        def hook(model, input, output):
            # Detach activations from computation graph and move to CPU
            activations[name].append(output.detach().cpu())
        return hook

    # Register hooks for each specified layer
    for name in layer_names:
        try:
            layer = dict([*model.named_modules()])[name]
            print(f"Layer {layer} is registered hook.")
            
        except KeyError:
            raise ValueError(f"Layer {name} not found in model.")
            
        hook = layer.register_forward_hook(get_activation(name))
        hooks.append(hook)

    # Handle large batch sizes by processing smaller batches of size 100
    batch_size = 100
    for i in range(0, image_tensor.size(0), batch_size):
        batch = image_tensor[i:i + batch_size]
        
        with torch.no_grad(): 
            model(batch)
        
    # Concatenate the activations for each layer across all batches
    for name in activations:
        activations[name] = torch.cat(activations[name], dim=0).numpy()
        # activations[name] = torch.cat(activations[name], dim=0)

    # Remove hooks after completion
    for hook in hooks:
        hook.remove()

    return activations


In [6]:
# Model dictionary, it maps the model names to their corresponding loading function and layers we want to extract activations from
model_dict = {
    'alexnet': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True),
        'layers': ["features.0", "features.2", "features.3", "features.5", "features.6", "features.8", "features.12", "classifier.1", "classifier.4"]
    },
    'vgg-16': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'vgg16', pretrained=True),
        'layers': ["features.4", "features.9", "features.16", "features.23", "features.30", "classifier.0", "classifier.3"]
    },
    'resnet-50': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True),
        'layers': ["layer1.0.bn1","layer1.1.bn1","layer1.2.bn1","layer2.0.bn1","layer2.1.bn1","layer2.2.bn1","layer3.1.bn1","layer3.2.bn1"]
    },
    'resnet-101': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True),
        'layers': ["layer3.2.bn1"]
    },
    'resnet-152': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=True),
        'layers': ["layer3.3.bn1"]
    },
    
    'densenet-121': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True),
        'layers': []
    },
    
    'densenet-201': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'densenet201', pretrained=True),
        'layers': []
    },

    'densenet-169': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'densenet169', pretrained=True),
        'layers': []
    },
    
    'squeezenet-1_0': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True),
        'layers': ["features.4.expand3x3","features.3.expand3x3","features.5.expand3x3","features.7.expand3x3","features.8.expand3x3","features.9.expand3x3","features.10.expand3x3","features.12.expand3x3"]
    },
    'squeezenet-1_1': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_1', pretrained=True),
        'layers': ["features.3.expand3x3_activation","features.4.expand3x3_activation","features.6.expand3x3_activation","features.7.expand3x3_activation","features.9.expand3x3_activation"]
    },
    'inception-v3': {
        'load': lambda: torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True),
        'layers': ["Mixed_7a.branch3x3_1.bn"]
    },
    'resnext-wsl': {
        'load': lambda: torch.hub.load('facebookresearch/WSL-Images', 'resnext101_32x8d_wsl'),
        'layers': ["layer3.3.relu"]
    },

}

In [7]:
# Define the device
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

# Define the folder path where the images are stored
images_folder_path = './data/images'

# Choose the models you want to extract from
models = ['resnet-50',
          'alexnet',
          'vgg-16' 
          'resnet-50',
          'resnet-101', 
          'resnet-152', 
          'densenet-121', 
          'densenet-201', 
          'densenet-169', 
          'squeezenet-1_0',
          'squeezenet-1_1',
          'inception-v3',
          'resnext-wsl']


In [ ]:
# Iterate through the models
for current_model in models: 
    
    model = model_dict[current_model]['load']()
    layers = model_dict[current_model]['layers']
    
    print(f"Loaded the model {current_model}")
    # Print the layer names (in case I want to see and update the layers)     
    for name, parameter in model.named_parameters():
        print(name)
       
    # Move the model to device and freeze the layers. 
    model.eval().to(device)
    for param in model.parameters():
        param.requires_grad = False
        
    # Initialize the tensors list. 
    img_tensors_list = []
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    
    # Iterate through the images and transform before passing to the model. 
    for i in range(1600):
            filename = f"im{i}.png"
            image_path = os.path.join(images_folder_path, filename)
            
            if os.path.exists(image_path):
                img = Image.open(image_path).convert("RGB")
                input_tensor = preprocess(img)
                input_batch = input_tensor.unsqueeze(0).to(device)
                img_tensors_list.append(input_batch)
            else:
                print(f"Warning: File {filename} not found. Skipping.")
                
    # Put the transformed images together and pass them to the model to get the activations.        
    big_tensor = torch.cat(img_tensors_list, dim=0)      
    activations = get_layer_activations(model, layers, big_tensor)
    
    # Save activations as pkl. 
    with open(f'./results/model_features/{current_model}_multiple_layers_features.pkl', 'wb') as f:
        pickle.dump(activations, f)
